# 실시간 Lip Landmark 추출


---
 
```꼭 확인하고 실험하기```

- **person**: 사람 
|code|담당|
|:----:|:----:|
|A, B, C|박희지|
|D, E, F|신은화|
|G, H, I|최영윤|
|J, K, L|최희수|
|M, N, O|한우정|

- **num**: 3회 반복횟수(00, 01, 02)
- **sentence**: 

|sentence code|sentence|비고|
|:---:|:----|----|
|00|어느쪽으로 가야해요?||
|01|오른쪽으로 가세요||
|02|왼쪽으로 가세요||
|03|앞으로 가세요||
|04|도와주세요||
|05|조심하세요||
|06|도와드릴까요?||
|07|다시 한번 말씀해주세요||
|08|안녕하세요||
|09|잘 지냈어?||
|10|다음에 봐||
|11|지금 뭐해?||
|12|죄송합니다||
|13|감사합니다||
|14|밥 먹었어?||
|15| 밥 먹자||

---

```실험 전```
- 사람&문장&반복횟수 **초기화**
----

```실험 동안```
- 코드 실행
- 코드에서 lip localization 한 값들이 뜨기 시작하면 시작
- 입을 벌리면 Frame 별 저장 시작
- 최대한 크게 최대한 가까이 최대한 밝은 공간에서 할 것!
- 얼굴은 정면으로
- 말은 조금은 느리게 (3초에 한단어씩이란 생각으로)
- 웹캠 끌때는 실행되고 있는 웹캠에서 'q' 눌러서 끄기
- 웹캠 반응이 없는데 계속 돌아가고 있다면, 위에 stop 눌러서 꺼주기
- 가끔 웹캠 반응 없으면 웹캠 종료 버튼 

---

```실험후```
- 사진들 저장되어있는거 확인
    - ***********No Detected******** 조심
- 앞이랑 뒤에 상관없는 사진들도 저장이 되었다면 삭제(입을 벌리기 직전 다물기 직전까지만 잘라서 저장: 수작업)
- crop사진들 확인 -> 얼굴 사진 저장되어있는 거랑 매치 시켜서 같이 저장되어있다면 **성공**
- 그렇지 않다면 **다시 찍기** -> 사진들 전부 폐기
- 결과 값 csv 파일로 저장

-  **커널 다시 돌리기**

In [6]:
import pandas as pd
import dlib
import cv2
import numpy as np
import os

In [7]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [8]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

In [13]:
######################################
# person & sentences
person = 'A' 
sentence= '00'
num = '01' 
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat") # predict 해주는애

# web cam start
video_capture = cv2.VideoCapture(0) 

while True:
    ret, frame = video_capture.read() #Get Frame: ret True, no: False

   # resize the video
    image = cv2.resize(frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)
    img = image.copy()
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get faces (up-sampling=1)
    face_detector = detector(img_gray, 1)
    
    if len(face_detector) == 0:
        print("***************No face Detected*************,%d", int(video_capture.get(1)))
    
    # for loop for faces
    for face in face_detector:
        # cropping face
        crop_face = image.copy()
        crop_face = crop_face[face.top(): face.bottom(), face.left(): face.right()]
        # make prediction and transform to numpy array
        landmarks = predictor(image, face)  # 얼굴에서 68개 점 찾기
        landmarks = shape_to_np(landmarks)
        lip_landmarks = [landmarks[x] for x in range(48, 68)]
        image_crop = image.copy()
        (x, y, w, h) = get_rect(lip_landmarks)
        crop_image = image_crop[y: y+h, x:x+w]
        
        for (i, (x_i, y_i)) in enumerate(lip_landmarks):
            cv2.circle(image, (x_i, y_i), 1, (255, 0, 0), -1)
            
        (x, y, w, h) = get_rect(lip_landmarks)
        
        for s in lip_landmarks:
            s[0] -= x
            s[1] -= y
    
        
    cv2.imshow('result', image)
    
    # exit camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    dataset[person][sentence][num][count] ={}   
    if (int(video_capture.get(1)) % 3 == 0): # after 5 sec passed (5초 단위)
        path = 'korea_data/%s/%s/%s/image'%(person, sentence, num)
        crop_path = 'korea_data/%s/%s/%s/crop_lip'%(person, sentence, num)
        crop_path2 = 'korea_data/%s/%s/%s/crop_face'%(person, sentence, num)
        
        #path 없다면 만들고 있다면 pass
        try:
            os.makedirs(path)
            os.makedirs(crop_path)
            os.makedirs(crop_path2)
        except FileExistsError as e:
            pass
        img_name = f'img_{count}.jpg'
        mouth_name = f'mouth_{count}.jpg'
        face_name = f'face_{count}.jpg'
        img_path = path + '/' + img_name
        mouth_path = crop_path + '/' + mouth_name
        face_path = crop_path2 + '/' + face_name
        
        # save images by frame
        cv2.imwrite(img_path, img)
        cv2.imwrite(mouth_path, crop_image) # 확인용
        cv2.imwrite(face_path, crop_face)
        
        print(lip_landmarks)
        print('Saved frame number : ' + str(int(video_capture.get(1)))+ "\n")
        dataset[person][sentence][num][count]= lip_landmarks  
        
        count += 1
        
        
        # exit camera
    if cv2.waitKey(1) == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

[array([ 0, 21]), array([ 6, 10]), array([14,  3]), array([20,  4]), array([26,  0]), array([37,  3]), array([48,  9]), array([41, 18]), array([33, 23]), array([26, 25]), array([19, 26]), array([10, 25]), array([ 5, 19]), array([15, 13]), array([22, 12]), array([28, 10]), array([43, 10]), array([30, 12]), array([23, 13]), array([17, 14])]
Saved frame number : 14430141

[array([ 0, 21]), array([ 5, 10]), array([13,  3]), array([20,  3]), array([26,  0]), array([36,  3]), array([47,  9]), array([40, 17]), array([32, 22]), array([25, 25]), array([18, 26]), array([ 9, 25]), array([ 4, 19]), array([15, 13]), array([21, 12]), array([27, 10]), array([43, 10]), array([29, 11]), array([23, 13]), array([16, 14])]
Saved frame number : 14430153

[array([ 0, 21]), array([ 6, 10]), array([13,  3]), array([20,  3]), array([25,  0]), array([36,  3]), array([47, 10]), array([40, 18]), array([32, 23]), array([25, 25]), array([18, 26]), array([ 9, 25]), array([ 5, 19]), array([15, 13]), array([21, 12]), 

In [5]:
#dataset

In [5]:
# csv 파일은 문장 폴더 안에 입모양 반복 횟수대로 이름지어서 저장됨
fileCount = 0
shapes = []
for count in dataset[person][sentence][num].keys():
    shape = dataset[person][sentence][num][count]
    for (x,y) in shape:
        shapes.append((num,count,x,y))

df = pd.DataFrame(data = shapes,columns = ['num','frame','x','y'])
path = "csv"
try:
    os.makedirs(path)
except FileExistsError as e:
    pass
filename = "%s_%s_%s.csv" %(person, sentence, num)
csvPath = "%s/%s" % (path,filename)
print("saving %s" % csvPath)
df.to_csv(csvPath)

saving csv/A_00_01.csv
